In [ ]:
############### Pytorch CIFAR configuration file ###############
import math
import functions.BinaryConnect as BC
import functions.DataAugmentation as DA
from functions.AutoAugment import AutoAugment, Cutout
from models.WideResnet_HRank import Wide_ResNet_HRank, wide_basic
from torch.nn import init
start_epoch = 1
num_epochs = 200
batch_size = 128
optim_type = 'SGD'

mean = {
    'cifar10': (0.4914, 0.4822, 0.4465),
    'cifar100': (0.5071, 0.4867, 0.4408),
}

std = {
    'cifar10': (0.2023, 0.1994, 0.2010),
    'cifar100': (0.2675, 0.2565, 0.2761),
}

# Only for cifar-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def learning_rate(init, epoch):
    optim_factor = 0
    if(epoch > 160):
        optim_factor = 4
    elif(epoch > 120):
        optim_factor = 2
    elif(epoch > 60):
        optim_factor = 1

    return init*math.pow(0.2, optim_factor)

def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

def fixedup(net):
    '''Init layer parameters.'''
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal_(m.weight, mode='fan_out')
            if m.bias != None:
                init.constant(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant_(m.weight, 1)
            init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal(m.weight, std=1e-3)
            if m.bias != None:
                init.constant_(m.bias, 0)
                
#Restricted Isometry
def l2_reg_ortho(mdl):
    l2_reg = None
    for W in mdl.parameters():
        if W.ndimension() < 2 :
            continue
        else:
            cols = W[0].numel()
            rows = W.shape[0]
            w1 = W.view(-1,cols)
            wt = torch.transpose(w1,0,1)
            if (rows > cols):
                m = torch.matmul(wt,w1)
                ident = Variable(torch.eye(cols,cols), requires_grad = True)
            else:
                m = torch.matmul(w1,wt)
                ident = Variable(torch.eye(rows,rows),requires_grad = True)
            
            ident = ident.cuda()
            w_tmp = (m-ident)
            b_k = Variable(torch.rand(w_tmp.shape[1],1))
            b_k = b_k.cuda()
            
            v1 = torch.matmul(w_tmp,b_k)
            norm1 = torch.norm(v1,2)
            v2 = torch.div(v1,norm1)
            v3 = torch.matmul(w_tmp,v2)
            
            if l2_reg is None:
                l2_reg = (torch.norm(v3,2))**2
            else:
                l2_reg = l2_reg + (torch.norm(v3,2))**2
    return l2_reg

def adjust_weight_decay_rate(optimizer,epoch,weight_decay=1e-4):
    w_d = weight_decay
    
    if epoch > 20:
        w_d = 5e-4
    elif epoch > 10:
        w_d = 1e-6
    
    for param_group in optimizer.param_groups:
        param_group['weight_decay'] = w_d

def adjust_ortho_decay_rate(epoch,ortho_decay=1e-2):
    o_d = ortho_decay
    if epoch > 120:
        o_d = 0.0
    elif epoch > 70:
        o_d = 1e-6*o_d
    elif epoch > 50:
        o_d = 1e-4*o_d
    elif epoch > 20:
        o_d = 1e-3*o_d
    return o_d

In [5]:
##### TRAINING CELL #####
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms
import numpy as np

import os
import sys
import time
import datetime


#Parameters settings
depth = 40 ##can be 10, 16, 22, 28(default), 34, 40
net_type = 'wide-resnet'
lr = 0.1
widen_factor = 2 #any numer, 10(default)
dropout = 0.3
dataset = 'cifar10'
testOnly = False
resume = True
bc = False
da = True
# Hyper Parameter settings
use_cuda = torch.cuda.is_available()
best_acc = 0

# Data Uplaod
print('\n[Phase 1] : Data Preparation')
if da:
    #print("*Using Data Augmentation")
    to_da = DA.DataAugmentation(dataset,aa=True, cut=True)
    if (dataset == 'cifar10'):
        num_classes = 10
    elif (dataset == 'cifar100'):
        num_classes = 100
    trainset_lenght,trainloader, testloader = to_da.load_data()
else:
    #print(\"| Using no Data Augmentation")
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean[dataset], std[dataset]),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean[dataset], std[dataset]),
    ])
    if(dataset == 'cifar10'):
        print("| Preparing CIFAR-10 dataset...")
        sys.stdout.write("| ")
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform = transform_test)
        trainset_length = len(trainset)
        num_classes = 10
    elif(dataset == 'cifar100'):
        print("| Preparing CIFAR-100 dataset...")
        sys.stdout.write("| ")
        trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = transform_train)
        testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform = transform_test)
        trainset_length = len(trainset)
        num_classes = 100
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

def mixup_data(x, y, alpha=0.2, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)





[Phase 1] : Data Preparation
| Preparing CIFAR-10 dataset...
Files already downloaded and verified
Files already downloaded and verified


In [3]:
def number_of_trainable_params(model):
        model_parameters = filter(lambda p: p.requires_grad, model.parameters())
        return sum([np.prod(p.size()) for p in model_parameters])

def saveList(myList,filename):
    # the filename should mention the extension 'npy'
    np.save(filename,myList)
    print("Saved successfully!")

def loadList(filename):
    # the filename should mention the extension 'npy'
    tempNumpyArray=np.load(filename)
    return tempNumpyArray.tolist()

In [7]:
from models.Wide_ResNet import Wide_ResNet

#net = Wide_ResNet_HRank(depth, widen_factor, dropout, num_classes)
net = Wide_ResNet(depth,widen_factor,dropout,num_classes)
conv_init(net)
print(number_of_trainable_params(net))
print('The weights have been initialized')
file_name = 'wide-resnet-srip-'+str(depth)+'x'+str(widen_factor)+str(dataset)


#for m in net.modules():
#    if isinstance(m,wide_basic):
#        m.pruning = False        

if bc:
    to_bc = BC(net)
    net = to_bc.model
    
# Test only option
if (testOnly):
    print('\n[Test Phase] : Model setup')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']

    if use_cuda:
        net.cuda()
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        cudnn.benchmark = True

    net.eval()
    net.training = False
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        acc = 100.*correct/total
        print("| Test Result\tAcc@1: %.2f%%" %(acc))

    sys.exit(0)

# Model
print('\n[Phase 2] : Model setup')
if(resume):
    # Load checkpoint
    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('| Building net type [' + net_type + ']...')
    net.apply(conv_init)

if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
    print('| Going fast AF with C U D A *o* !')

criterion = nn.CrossEntropyLoss()

# Training
def train(epoch,odecay):
    net.train()
    net.training = True
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=learning_rate(lr, epoch), momentum=0.9, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate(lr, epoch)))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        
        #inputs, targets_a,targets_b,lam = mixup_data(inputs,targets)
        
        #inputs, targets_a,targets_b = map(Variable,(inputs,targets_a,targets_b))

        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        if bc:
            bc.binarization()
            outputs = net(inputs)       # Forward Propagation\n",
            loss = criterion(outputs,targets)
            bc.restore()
            loss.backward()
            bc.clip()
            optimizer.step()
        else:
            outputs = net(inputs)
            
            oloss = l2_reg_ortho(net)
            oloss = odecay*oloss
            loss = criterion(outputs,targets)
            loss = loss + oloss
            #loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

            
        train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)

        total += targets.size(0)
        
        #correct += (lam * predicted.eq(targets_a.data).cpu().sum().float()+ (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float())
        correct += predicted.eq(targets.data).cpu().sum()
        loss.backward()
        optimizer.step()

        sys.stdout.write('\r')

        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (trainset_lenght//batch_size)+1, loss.item(), 100.*correct/total))
        sys.stdout.flush()

def test(epoch):
    global best_acc
    net.eval()
    net.training = False
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        # Save checkpoint when best model
        acc = 100.*correct/total
        print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))

        if acc > best_acc:
            print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
            state = {
                    'net':net.module if use_cuda else net,
                    'acc':acc,
                    'epoch':epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            save_point = './checkpoint/'+dataset+os.sep
            if not os.path.isdir(save_point):
                os.mkdir(save_point)
            torch.save(state, save_point+file_name+'.t7')
            best_acc = acc

print('\n[Phase 3] : Training model')
print('| Training Epochs = ' + str(num_epochs))
print('| Initial Learning Rate = ' + str(lr))
print('| Optimizer = ' + str(optim_type))

elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()
    odecay = adjust_ortho_decay_rate(epoch+1)
    train(epoch,odecay)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))
torch.save(net,"wide_resnet.pth")
print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))

| Wide-Resnet 40x2
2246474
The weights have been initialized

[Phase 2] : Model setup
| Resuming from checkpoint...
| Going fast AF with C U D A *o* !

[Phase 3] : Training model
| Training Epochs = 200
| Initial Learning Rate = 0.1
| Optimizer = SGD

=> Training Epoch #149, LR=0.0040
| Epoch [149/200] Iter[391/391]		Loss: 0.1998 Acc@1: 92.340%
| Validation Epoch #149			Loss: 0.0804 Acc@1: 94.76%
| Elapsed time : 0:01:08

=> Training Epoch #150, LR=0.0040
| Epoch [150/200] Iter[391/391]		Loss: 0.1180 Acc@1: 92.474%
| Validation Epoch #150			Loss: 0.0091 Acc@1: 94.70%
| Elapsed time : 0:02:16

=> Training Epoch #151, LR=0.0040


KeyboardInterrupt: 

In [0]:
torch.save(net,"wide-resnet-xavier-mixup-40x2.pth")

In [8]:
#Install the module torch_pruning
!pip3 install torch_pruning

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 753.4MB 2.1kB/s eta 0:00:011  10% |███▌                            | 82.8MB 44.1MB/s eta 0:00:16    12% |████                            | 96.4MB 44.8MB/s eta 0:00:15    17% |█████▌                          | 129.4MB 48.2MB/s eta 0:00:13    23% |███████▍                        | 173.6MB 56.5MB/s eta 0:00:11    24% |████████                        | 186.3MB 55.8MB/s eta 0:00:11    35% |███████████▎                    | 266.6MB 41.4MB/s eta 0:00:12    52% |████████████████▊               | 392.6MB 35.1MB/s eta 0:00:11    61% |███████████████████▊            | 464.5MB 25.7MB/s eta 0:00:12    69% |██████████████████████▎         | 525.2MB 45.7MB/s eta 0:00:05    97% |███████████████████████████████▏| 733.7MB 23.7MB/s eta 0:00:01


In [10]:
#Computer params/flops of a given net
!pip install --upgrade git+https://github.com/sovrasov/flops-counter.pytorch.git

  Cloning https://github.com/sovrasov/flops-counter.pytorch.git to /tmp/pip-req-build-o8bme1q9
  Running command git clone -q https://github.com/sovrasov/flops-counter.pytorch.git /tmp/pip-req-build-o8bme1q9
  Created wheel for ptflops: filename=ptflops-0.5.2-cp37-none-any.whl size=8220 sha256=7fce7037f840e6df6f1cb2472ad8dd3a8b1e2d14fc7ad7b569feec116bf92523
  Stored in directory: /tmp/pip-ephem-wheel-cache-cbyx1g1d/wheels/00/ce/d1/169969eba40b2078b42c637bc9aac0f265e75a8a951b4e8570
Successfully built ptflops


In [12]:
import torch
from ptflops import get_model_complexity_info


with torch.cuda.device(0):
  flops, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))



DataParallel(
  36.547 M, 100.000% Params, 5.959 GMac, 100.000% MACs, 
  (module): Wide_ResNet(
    36.547 M, 100.000% Params, 5.959 GMac, 100.000% MACs, 
    (conv1): Conv2d(0.0 M, 0.001% Params, 0.0 GMac, 0.008% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (layer1): Sequential(
      1.642 M, 4.493% Params, 1.682 GMac, 28.216% MACs, 
      (0): wide_basic(
        0.257 M, 0.703% Params, 0.263 GMac, 4.413% MACs, 
        (bn1): BatchNorm2d(0.0 M, 0.000% Params, 0.0 GMac, 0.001% MACs, 16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(0.023 M, 0.063% Params, 0.024 GMac, 0.399% MACs, 16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (dropout): Dropout(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, p=0.3, inplace=False)
        (bn2): BatchNorm2d(0.0 M, 0.001% Params, 0.0 GMac, 0.005% MACs, 160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(0.231 M, 0.631% Params, 0.

In [0]:
#Fine Tuning
def learning_rate_ft(init, epoch):
    optim_factor = 0
    if(epoch > 3):
        optim_factor = 3
    elif(epoch > 2):
        optim_factor = 2
    elif(epoch > 1):
        optim_factor = 1

    return init*math.pow(0.2, optim_factor)
    
def fine_tuning_train(epoch,net,bc = False, num_epochs = 3, lr = 0.001):
    net.train()
    net.training = True
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=learning_rate_ft(lr, epoch), momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    print('\n => Fine Tuning Epoch #%d, LR=%.4f' %(epoch, learning_rate_ft(lr, epoch)))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        if bc:
            bc.binarization()
            outputs = net(inputs)       # Forward Propagation\n",
            loss = criterion(outputs,targets)
            bc.restore()
            loss.backward()
            bc.clip()
            optimizer.step()
        else:
            outputs = net(inputs)
            loss = criterion(outputs,targets)
            loss.backward()
            optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (trainset_lenght//batch_size)+1, loss.item(), 100.*correct/total))
        sys.stdout.flush()

def fine_tuning_test(epoch,net):
    global best_acc
    net.eval()
    net.training = False
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        # Save checkpoint when best model
        acc = 100.*correct/total
        if epoch != 0:
          print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))

        if acc > best_acc:
            print('| New Best Accuracy...\t\t\tTop1 = %.2f%%' %(acc))
            print('| Saving Pruned Model...')
            torch.save(net,"wide_resnet.pth")
            best_acc = acc

In [0]:
#Hard Pruning
from functions.HardPruning import HardPrunning
from functions.HRankPruning import HRank
from models.Wide_ResNet import Wide_ResNet
import torch_pruning as pruning
import torch
import os
import numpy as np

import matplotlib.pyplot as plt

import time

#Params
depth = 40
net_type = 'wide-resnet'
widen_factor = 2 
dropout = 0.3
dataset = 'cifar10'
num_classes = 10

file_name = 'wide-resnet-'+str(depth)+'x'+str(widen_factor)


#WideResnet 40x2 has 3 layers
net = Wide_ResNet(depth, widen_factor, dropout, num_classes)

#Pruning ratios for each layer
pruning_ratios_layer1 = [[x,0.0,0.0] for x in np.linspace(0,0.9,10)]
pruning_ratios_layer2 = [[0.0,x,0.0] for x in np.linspace(0,0.9,10)]
pruning_ratios_layer3 = [[0.0,0.0,x] for x in np.linspace(0,0.9,10)]
pruning_ratios = [x for x in np.linspace(0,0.9,10)]

#Pruning based on the score of the filters
fscore_accuracy_layer1 = []
fscore_accuracy_layer2 = []
fscore_accuracy_layer3 = []
fscore_net_weights_layer1 = []
fscore_net_weights_layer2 = []
fscore_net_weights_layer3 = []

with torch.cuda.device(0):
  flops, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True, print_per_layer_stat=False)
  print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

for r in pruning_ratios_layer3:
    # Load checkpoint
    

    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']
    best_acc = -1000
    print('\n[1] PRUNING-----------------------------------------------------------')
    print('\n=> Pruning Layer 2... | Ratio : {}%'.format(r[2]))
    fscore = HardPrunning(net,r)
    fscore.HardPruning()
    removed_weights = number_of_trainable_params(fscore.model)
    weights_diff = initial_weights - removed_weights

    print('| Weights removed : {} | {}%'.format(weights_diff,int(100 - 100*removed_weights/initial_weights)))
    fscore_net_weights_layer3.append(removed_weights)

    print('\n[2] FINE TUNING-------------------------------------------------------')
    elapsed_time = 0
    for epoch in range(1, 4):
        start_time = time.time()

        fine_tuning_train(epoch,fscore.model)
        fine_tuning_test(epoch, fscore.model)

        epoch_time = time.time() - start_time
        elapsed_time += epoch_time
        print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

    print('\n[3] TESTING -----------------------------------------------------------')
    print('Testing model..')
    print('* Test results : Acc@1 = %.2f%%' %(best_acc))
    print('\n ----------------------------------------------------------------------')
    fscore_accuracy_layer3.append(best_acc.item())
    best_acc = -1000

saveList(fscore_accuracy_layer3,"fscore_accuracy_layer3")
saveList(fscore_net_weights_layer3,"score_net_weights_layer3")



plt.plot(loadList("score_net_weights_layer3.npy"),loadList("fscore_accuracy_layer3.npy"),)
plt.show()




In [0]:
#Hard Rank Pruning
from functions.HRankPruning import HRank
from models.WideResnet_HRank import Wide_ResNet_HRank
import torch_pruning as pruning
import torch
import os
import numpy as np

from torchvision.datasets import CIFAR10
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import time

#Params
depth = 40
net_type = 'wide-resnet'
widen_factor = 2 
dropout = 0.3
dataset = 'cifar10'
num_classes = 10

file_name = 'wide-resnet-'+str(depth)+'x'+str(widen_factor)


#WideResnet 40x2 has 3 layers
net = Wide_ResNet_HRank(depth, widen_factor, dropout, num_classes)
net.cuda()
    
#Data for net analysis
def generate_subset(dataset,n_classes,reducefactor,n_ex_class_init):
    nb_examples_per_class = int(np.floor(n_ex_class_init / reducefactor))
    # Generate the indices. They are the same for each class, could easily be modified to have different ones. But be careful to keep the random seed! 
    indices_split = np.random.RandomState(seed=42).choice(n_ex_class_init,nb_examples_per_class,replace=False)
    all_indices = []
    for curclas in range(n_classes):
        curtargets = np.where(np.array(dataset.targets) == curclas)
        indices_curclas = curtargets[0]
        indices_subset = indices_curclas[indices_split]
        #print(len(indices_subset))
        all_indices.append(indices_subset)
    all_indices = np.hstack(all_indices)
    return Subset(dataset,indices=all_indices)

rootdir = './data/'
normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)
subset = generate_subset(dataset=c10test,n_classes=10,reducefactor=20,n_ex_class_init=1000)
loader = DataLoader(subset,batch_size=50, num_workers=2)

#Pruning ratios for each layer
pruning_ratios_layer1 = [[x,0.0,0.0] for x in np.linspace(0,0.9,10)]
pruning_ratios_layer2 = [[0.0,x,0.0] for x in np.linspace(0,0.9,10)]
pruning_ratios_layer3 = [[0.0,0.0,x] for x in np.linspace(0,0.9,10)]
pruning_ratios = [x for x in np.linspace(0,0.9,10)]

#Pruning based on the score of the filters
hscore_accuracy_layer1 = []
hscore_accuracy_layer2 = []
hscore_accuracy_layer3 = []
hscore_net_weights_layer1 = []
hscore_net_weights_layer2 = []
hscore_net_weights_layer3 = []

initial_weights = number_of_trainable_params(net)

print('[ Weights : {}]'.format(initial_weights))

for r in pruning_ratios_layer3:
    # Load checkpoint
    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']
    best_acc = -1000

    print('\n[1] PRUNING-----------------------------------------------------------')
    print('\n=> Pruning Net... | Layer1 : {}% Layer2 : {}% Layer3 : {}%'.format(r[0]*100,r[1]*100,r[2]*100))
    for m in net.modules():
        if isinstance(m,wide_basic):
            m.pruning = True  
    hscore = HRank(net,loader,r)
    hscore.HRank()
    removed_weights = number_of_trainable_params(hscore.model)
    weights_diff = initial_weights - removed_weights
    print('| Weights removed : {} | {}%'.format(weights_diff,int(100 - 100*removed_weights/initial_weights)))
    hscore_net_weights_layer3.append(removed_weights)

    print('\n[2] FINE TUNING-------------------------------------------------------')
    elapsed_time = 0
    for m in net.modules():
        if isinstance(m,wide_basic):
            m.pruning = False  
    for epoch in range(1, 4):
        start_time = time.time()

        fine_tuning_train(epoch,hscore.model)
        fine_tuning_test(epoch,hscore.model)

        epoch_time = time.time() - start_time
        elapsed_time += epoch_time
        print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

    print('\n[3] TESTING -----------------------------------------------------------')
    print('Testing model..')
    print('* Test results : Acc@1 = %.2f%%' %(best_acc))
    print('\n ----------------------------------------------------------------------')
    hscore_accuracy_layer3.append(best_acc.item())
    best_acc = -1000

saveList(hscore_accuracy_layer3,"hscore_accuracy_layer3")
saveList(hscore_net_weights_layer3,"hscore_net_weights_layer3")



plt.plot(loadList("hscore_net_weights_layer3.npy"),loadList("hscore_accuracy_layer3.npy"),)
plt.show()



In [0]:
#Iterative pruning
#Method : Hard Pruning based on the score of the filters
from functions.HardPruningIter import HardPrunningIter
from models.Wide_ResNet import Wide_ResNet
import torch_pruning as pruning
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import time


#Params
depth = 40
net_type = 'wide-resnet'
widen_factor = 2 
dropout = 0.3
dataset = 'cifar10'
num_classes = 10
file_name = 'wide-resnet-'+str(depth)+'x'+str(widen_factor)

#WideResnet 40x2 has 3 layers
net = Wide_ResNet(depth, widen_factor, dropout, num_classes)

#Pruning ratios for each layer
#pruning_ratios_layer1 = [[x,0.0,0.0] for x in np.linspace(0,0.9,10)]
pruning_ratios_layer2 = [[0.0,x,0.0] for x in [0.95]]
pruning_ratios_layer3 = [[0.0,0.0,x] for x in [0.95]]
pruning_ratios = [x for x in np.linspace(0,0.9,10)]

#Store the results
fscore_iter_accuracy_layer1 = []
fscore_iter_accuracy_layer2 = []
fscore_iter_accuracy_layer3 = []
score_iter_net_weights_layer1 = []
score_iter_net_weights_layer2 = []
score_iter_net_weights_layer3 = []
test = [[0.0,0.0,0.05],[0,0,0.10]]

for r in pruning_ratios_layer2:
    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']

    fscore_iter = HardPrunningIter(net,r)
    fscore_iter.pruning_and_training(testloader,trainloader,epoch=3)

with torch.cuda.device(0):
  flops, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

fscore_iter_accuracy_layer3 = [95.36,95.49,95.33,95.41,95.48,95.33,95.26,95.36,95.42,95.17,94.92,95.05,94.90,94.67,94.62,94.34,94.48,94.18,93.85,93.59,92.91,92.51,90.58]
score_iter_net_weights_layer3 = [2180144,2113814,2047484,1981154,1914824,1848494,1782164,1715834,1649504,1583174,1516844,1450514,1384184,1314854,1251524,1185194,1118864,1052534,986204,853544,787214,720884,654554]


Trainable params [2.0 M]| Iteration [  4] Epoch [  3/  3] Iter [391/391] LR [  0] 		Loss: 0.2496 Acc@1: 93.488%
 | Test 94.7699966430664 
| New Best Accuracy...			Top1 = 94.77%
| Saving Pruned Model...

[1] PRUNING | ITER : 5/12-----------------------------------------------------------

=> Pruning Net... | Layer1 : 0.0% Layer2 : 95.0% Layer3 : 0.0%
Computational complexity:       0.28 GMac
Number of parameters:           1.95 M  

[2] FINE TUNING----------------------------------------------------------------------
Trainable params [1.95 M]| Iteration [  5] Epoch [  1/  3] Iter [391/391] LR [  0] 		Loss: 0.3289 Acc@1: 91.216%
 | Test 93.18000030517578 
| New Best Accuracy...			Top1 = 93.18%
| Saving Pruned Model...
Trainable params [1.95 M]| Iteration [  5] Epoch [  2/  3] Iter [391/391] LR [  0] 		Loss: 0.1904 Acc@1: 91.636%
 | Test 94.19999694824219 
| New Best Accuracy...			Top1 = 94.20%
| Saving Pruned Model...
Trainable params [1.95 M]| Iteration [  5] Epoch [  3/  3] Iter [391/3

In [0]:
#Iterative pruning HRank
from functions.HRankPruningIter import HRankIter

from models.WideResnet_HRank import Wide_ResNet_HRank
import torch_pruning as pruning
import torch
import os
import numpy as np

from torchvision.datasets import CIFAR10
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

import time

#Params
depth = 40
net_type = 'wide-resnet'
widen_factor = 2 
dropout = 0.3
dataset = 'cifar10'
num_classes = 10

file_name = 'wide-resnet-'+str(depth)+'x'+str(widen_factor)


#WideResnet 40x2 has 3 layers
net = Wide_ResNet_HRank(depth, widen_factor, dropout, num_classes)
net.cuda()
for m in net.modules():
        if isinstance(m,wide_basic):
            m.pruning = True  
  
#Data for net analysis
def generate_subset(dataset,n_classes,reducefactor,n_ex_class_init):
    nb_examples_per_class = int(np.floor(n_ex_class_init / reducefactor))
    # Generate the indices. They are the same for each class, could easily be modified to have different ones. But be careful to keep the random seed! 
    indices_split = np.random.RandomState(seed=42).choice(n_ex_class_init,nb_examples_per_class,replace=False)
    all_indices = []
    for curclas in range(n_classes):
        curtargets = np.where(np.array(dataset.targets) == curclas)
        indices_curclas = curtargets[0]
        indices_subset = indices_curclas[indices_split]
        #print(len(indices_subset))
        all_indices.append(indices_subset)
    all_indices = np.hstack(all_indices)
    return Subset(dataset,indices=all_indices)

rootdir = './data/'
normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)
subset = generate_subset(dataset=c10test,n_classes=10,reducefactor=5,n_ex_class_init=1000)
loader = DataLoader(subset,batch_size=32, num_workers=2)

#Pruning ratios for each layer
pruning_ratios_layer1 = [[0.95,0.0,0.0]]
pruning_ratios_layer2 = [[0.0,0.95,0.0]]
pruning_ratios_layer3 = [[0.0,0.0,0.95]]
pruning_ratios = [x for x in np.linspace(0,0.9,10)]

initial_weights = number_of_trainable_params(net)

print('[ Weights : {}]'.format(initial_weights))
for r in pruning_ratios_layer3:
    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']

    hscore_iter = HRankIter(net,r,5)
    hscore_iter.pruning_and_training(loader,trainloader,testloader,epoch=3)


In [0]:
#Save the weights of the pruned net
model = 'wide_resnet_40x2_pruned_cifar10.pth'
torch.save(net,model)

In [0]:
#Functions for advanced fine tuning
def ft_lr(init, epoch):
    optim_factor = 0
    if(epoch > 9):
        optim_factor = 3
    elif(epoch > 6):
        optim_factor = 2
    elif(epoch > 3):
        optim_factor = 1

    return init*math.pow(0.2, optim_factor)
    
def ft_train(epoch,net,bc = False, num_epochs = 140, lr = 0.1):
    net.train()
    net.training = True
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=ft_lr(lr, epoch), momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    print('\n => Fine Tuning Epoch #%d/%d, LR=%.4f' %(epoch,num_epochs, ft_lr(lr, epoch)))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        if bc:
            bc.binarization()
            outputs = net(inputs)       # Forward Propagation\n",
            loss = criterion(outputs,targets)
            bc.restore()
            loss.backward()
            bc.clip()
            optimizer.step()
        else:
            outputs = net(inputs)
            loss = criterion(outputs,targets)
            loss.backward()
            optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (trainset_lenght//batch_size)+1, loss.item(), 100.*correct/total))
        sys.stdout.flush()

def ft_test(epoch,net):
    global best_acc
    net.eval()
    net.training = False
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        # Save checkpoint when best model
        acc = 100.*correct/total
        if epoch != 0:
          print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))

        if acc > best_acc:
            print('| New Best Accuracy...\t\t\tTop1 = %.2f%%' %(acc))
            print('| Saving Model...')
            torch.save(net,"wide_resnet_40x2_pruned_trained_cifar10.pth")
            best_acc = acc

In [0]:
#Fine Tuning with 50 epochs with the pruned model
#The goal is to reach a better accuracy after the pruning

#Params
num_epochs = 10
lr = 0.0002
start_epoch = 1
best_acc = 0
        
print('\n ADVANCED FINE TUNING-------------------------------------------------------')
print('| Fine Tuning Epochs = ' + str(num_epochs))
print('| Initial Learning Rate = ' + str(lr))
print('| Optimizer = ' + 'SGD')

#Load the pruned model
model = 'wide_resnet_40x2_pruned_cifar10.pth'
brain = torch.load(model)
brain.cuda()

#Advanced fine tuning
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    ft_train(epoch,brain,num_epochs = num_epochs,lr = lr)
    ft_test(epoch,brain)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))



 ADVANCED FINE TUNING-------------------------------------------------------
| Fine Tuning Epochs = 10
| Initial Learning Rate = 0.0002
| Optimizer = SGD

 => Fine Tuning Epoch #1/10, LR=0.0002
| Epoch [  1/ 10] Iter[261/391]		Loss: 0.3077 Acc@1: 86.434%

KeyboardInterrupt: ignored